In [1]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import os
import numpy as np

# Define the transformations to apply to the images.
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),  # Randomly crop the images to 224x224.
    transforms.RandomHorizontalFlip(),  # Randomly flip the images horizontally.
    transforms.ToTensor(),  # Convert the images to PyTorch tensors.
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with the given mean and std.
])

# Path to the dataset directory.
data_dir = 'archive'

# Load the dataset from the directory applying the defined transformations.
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split the dataset into training and validation sets.
train_size = int(0.8 * len(dataset))  # 80% of the dataset for training.
val_size = len(dataset) - train_size  # The rest for validation.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])  # Perform the split.

# Create data loaders for training and validation datasets.
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)  # For training.
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)  # For validation.

print(f"Training dataset size: {len(train_dataset)}")  # Print the size of the training dataset.
print(f"Validation dataset size: {len(val_dataset)}")  # Print the size of the validation dataset.


Training dataset size: 10252
Validation dataset size: 2563


In [2]:
import torch
import torch.nn as nn
from torchvision import models

# Define an emotion classification model
class EmotionClassifier(nn.Module):
    def __init__(self, num_classes=7):  # Assume there are 7 different emotions to classify
        super(EmotionClassifier, self).__init__()
        # Load a pre-trained DenseNet121 model
        self.densenet = models.densenet121(pretrained=True)
        # Replace the classifier part to fit our new classification task
        num_ftrs = self.densenet.classifier.in_features
        self.densenet.classifier = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        # Define the forward pass through the network
        return self.densenet(x)

# Assuming you're using a GPU
model = EmotionClassifier().to('cuda')  # Move the model to CUDA (GPU)


/home/vopo/anaconda3/envs/pytorch_env1/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/vopo/anaconda3/envs/pytorch_env1/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# Initialize the loss function
# CrossEntropyLoss is commonly used for classification tasks. It combines softmax and 
# negative log likelihood loss in one class, making it suitable for multi-class classification.
criterion = nn.CrossEntropyLoss()

# Initialize the optimizer
# Adam optimizer is chosen for its efficiency with large datasets and high-dimensional spaces.
# It adjusts the learning rate dynamically. Here, we set an initial learning rate of 0.001.
# The optimizer updates the model parameters (weights) during the training process.
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [4]:
class EarlyStopping:
    """Implements early stopping to prevent overfitting during training."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Initializes the EarlyStopping instance.
        
        Args:
            patience (int): The number of epochs with no improvement after which training will be stopped.
            verbose (bool): If True, prints a message each time the early stopping criterion is updated.
            delta (float): The minimum change in the monitored quantity to qualify as an improvement.
            path (str): The file path to save the model upon improvement.
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0  # Tracks how many epochs have gone by without improvement
        self.best_score = None  # The best score seen so far
        self.early_stop = False  # Indicates whether early stopping condition was met
        self.val_loss_min = np.Inf  # Initialize the minimum validation loss observed as infinity
        self.delta = delta
        self.path = path  # Model save path

    def __call__(self, val_loss, model):
        """Checks whether the validation loss has improved and decides whether to stop early."""
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0  # Reset the improvement counter

    def save_checkpoint(self, val_loss, model):
        """Saves the model when the validation loss decrease."""
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...')
        torch.save(model.state_dict(), self.path)  # Save the model
        self.val_loss_min = val_loss  # Update the minimum validation loss


In [5]:
def train_model(model, criterion, optimizer, num_epochs=25):
    # Initialize the EarlyStopping object
    early_stopping = EarlyStopping(patience=10, verbose=True, path='best_model.pt')
    
    for epoch in range(num_epochs):  # Loop over the dataset multiple times
        model.train()  # Set model to training mode
        running_loss = 0.0  # To accumulate the loss over the epoch
        
        # Iterate over the training data.
        for inputs, labels in train_loader:  # Get the inputs; data is a list of [inputs, labels]
            inputs, labels = inputs.to('cuda'), labels.to('cuda')  # Move inputs and labels to the GPU if available

            optimizer.zero_grad()  # Zero the parameter gradients to avoid accumulation

            # Forward pass: compute the output of the network by passing inputs through the model
            outputs = model(inputs)
            loss = criterion(outputs, labels)  # Compute the loss

            # Backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            optimizer.step()  # Perform a single optimization step (parameter update)

            running_loss += loss.item() * inputs.size(0)  # Multiply the loss by the batch size for normalization
        
        epoch_loss = running_loss / len(train_loader.dataset)  # Calculate the average loss over the epoch
        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}')

        # Validation phase: Evaluate the model on the validation data
        model.eval()  # Set the model to evaluation mode
        val_running_loss = 0.0
        corrects = 0  # To accumulate the number of correctly classified examples

        with torch.no_grad():  # Disabling gradient calculation
            for inputs, labels in val_loader:
                inputs, labels = inputs.to('cuda'), labels.to('cuda')
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)  # Get the predictions
                corrects += torch.sum(preds == labels.data)  # Count correct predictions

        val_loss = val_running_loss / len(val_loader.dataset)  # Calculate the average validation loss
        val_acc = corrects.double() / len(val_loader.dataset)  # Calculate the accuracy

        print(f'Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

        # Check for early stopping
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break  # Exit the training loop

    # Load the best model weights
    model.load_state_dict(torch.load('best_model.pt'))

    return model  # Return the trained model

# Start the training process
model = train_model(model, criterion, optimizer, num_epochs=25)


Epoch 0/24, Loss: 1.6150
Val Loss: 1.5735 Acc: 0.3597
Validation loss decreased (inf --> 1.573479).  Saving model ...
Epoch 1/24, Loss: 1.4233
Val Loss: 1.3976 Acc: 0.4144
Validation loss decreased (1.573479 --> 1.397641).  Saving model ...
Epoch 2/24, Loss: 1.3324
Val Loss: 1.4269 Acc: 0.4058
EarlyStopping counter: 1 out of 10
Epoch 3/24, Loss: 1.2956
Val Loss: 1.2920 Acc: 0.4596
Validation loss decreased (1.397641 --> 1.292019).  Saving model ...
Epoch 4/24, Loss: 1.2723
Val Loss: 1.4021 Acc: 0.4428
EarlyStopping counter: 1 out of 10
Epoch 5/24, Loss: 1.2544
Val Loss: 1.3608 Acc: 0.4343
EarlyStopping counter: 2 out of 10
Epoch 6/24, Loss: 1.2249
Val Loss: 1.2519 Acc: 0.4686
Validation loss decreased (1.292019 --> 1.251855).  Saving model ...
Epoch 7/24, Loss: 1.2187
Val Loss: 1.2387 Acc: 0.5014
Validation loss decreased (1.251855 --> 1.238728).  Saving model ...
Epoch 8/24, Loss: 1.1937
Val Loss: 1.1942 Acc: 0.5185
Validation loss decreased (1.238728 --> 1.194167).  Saving model ...


In [9]:
from torchvision import transforms
from PIL import Image
import torch

# Define the image preprocessing steps
transform = transforms.Compose([
    transforms.Resize(256),  # Resize the image to 256 pixels on the shorter side
    transforms.CenterCrop(224),  # Crop a square of 224x224 pixels from the center of the image
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image with the ImageNet mean and standard deviation
])

# Load and preprocess an image
def load_image(image_path):
    image = Image.open(image_path)  # Open the image file
    # Apply the preprocessing pipeline
    image = transform(image).unsqueeze(0)  # Add a batch dimension to the tensor
    return image

# Predict the emotion from an image
def predict_emotion(model, image_path):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Do not compute or store gradients, saving memory and speeding up prediction
        image = load_image(image_path)
        image = image.to('cuda')  # Move the image tensor to the GPU if available
        outputs = model(image)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        return probabilities.cpu().numpy()  # Move the probabilities back to CPU memory if using GPU

# Use the model to make a prediction
image_path = '13.png'  # Replace with the path to your image
probabilities = predict_emotion(model, image_path)

# Print the probabilities
print("Probabilities:", probabilities)

# To associate each probability with its corresponding emotion label:
emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
predicted_emotion = emotion_labels[probabilities.argmax()]  # Find the emotion with the highest probability
print(f"Predicted Emotion: {predicted_emotion} with probability {probabilities.max()}")


Probabilities: [[0.0172285  0.07257926 0.13461143 0.02390061 0.22742681 0.01419428
  0.51005906]]
Predicted Emotion: surprise with probability 0.5100590586662292
